### Download Data - requires Google SDK tool GSUTIL

In [ ]:
gsutil -m rsync -r gs://nmfs_odp_pifsc/PIFSC/SOD/MOUSS/jpg/20161014_192048_1 U:\temp\20161014_192048_1

# Yolo8n Training -
## Unsupervised training of Yolo8n (nano) model with Yolo8x large model and dataset

In [ ]:
import os
import torch
from ultralytics import YOLO

unzipped_folder = r"V:\temp\20161014_192048_1"  # Use raw string for Windows paths
print(torch.cuda.is_available())  # Should return True if CUDA is properly set up
print(torch.version.cuda)  # Prints the version of CUDA PyTorch is using
print(f"Available GPUs: {torch.cuda.device_count()}")
print(f"Current GPU: {torch.cuda.current_device()}")
print(f"GPU Name: {torch.cuda.get_device_name(torch.cuda.current_device())}")

# Step 3: Check if CUDA is available and load the large model (YOLOv8x) to CUDA if possible
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load the large model (YOLOv8x) that was fine-tuned for VME detection
large_model = YOLO(r"V:\temp\models\best.pt")  # Update to your model path
large_model = large_model.to(device)  # Move the model to GPU (CUDA) if available

# Step 4: Create folders for the new dataset (images and labels)
base_path = r"V:\temp\lg_fish_dataset"

# Create directories for train/val images and labels
os.makedirs(os.path.join(base_path, "images", "train"), exist_ok=True)
os.makedirs(os.path.join(base_path, "images", "val"), exist_ok=True)
os.makedirs(os.path.join(base_path, "labels", "train"), exist_ok=True)
os.makedirs(os.path.join(base_path, "labels", "val"), exist_ok=True)

# Step 5: Define the input folder containing the original images (unzipped)
input_images_folder = unzipped_folder
image_paths = [os.path.join(input_images_folder, img) for img in os.listdir(input_images_folder) if img.endswith(('.jpg', '.png'))]

# Split images into train and validation (80% train, 20% validation)
train_images = image_paths[:int(0.8 * len(image_paths))]
val_images = image_paths[int(0.8 * len(image_paths)):]

# Function to save YOLO format labels (class_id x_center y_center width height)
def save_yolo_labels(label_path, class_id, bbox, image_width, image_height):
    if bbox is not None and len(bbox) > 0:  # Ensure there are bounding boxes
        x_center = (bbox[0] + bbox[2]) / 2 / image_width
        y_center = (bbox[1] + bbox[3]) / 2 / image_height
        width = (bbox[2] - bbox[0]) / image_width
        height = (bbox[3] - bbox[1]) / image_height
        with open(label_path, "a") as f:
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
        print(f"Label saved for {label_path}: {class_id} {x_center} {y_center} {width} {height}")
    else:
        print(f"No bounding boxes found for {label_path}, skipping label.")

# Function to process images, run inference, and save results
def process_images(image_paths, split):
    for image_path in image_paths:
        # Step 6: Run the large model to detect objects in the image (with GPU if available)
        results = large_model(image_path)

        # Check if any instances were detected
        print(f"Processing {image_path}, found {len(results[0].boxes)} instances.")

        # Extract image dimensions
        img_name = os.path.basename(image_path)
        img = results[0].orig_img
        img_height, img_width = img.shape[:2]

        # Save the image to the appropriate split folder (train/val)
        output_image_path = os.path.join(base_path, "images", split, img_name)
        os.rename(image_path, output_image_path)

        # Step 7: Filter results to only include "fish" category (class_id = 2)
        fish_class_index = 2  # Assuming fish is class 2 in the large model
        for result in results:
            for i, cls in enumerate(result.boxes.cls):
                if cls == fish_class_index:  # Only keep fish detections
                    bbox = result.boxes.xyxy[i].cpu().numpy()  # Bounding box (x1, y1, x2, y2)
                    print(f"Detected fish with bounding box: {bbox}")
                    print(f"Image dimensions: {img_width}x{img_height}")

                    # Step 8: Save label file in YOLO format
                    label_name = img_name.replace(".jpg", ".txt").replace(".png", ".txt")
                    label_path = os.path.join(base_path, "labels", split, label_name)

                    # Save the bounding box in YOLO format (class_id = 0 for fish)
                    save_yolo_labels(label_path, class_id=0, bbox=bbox, image_width=img_width, image_height=img_height)

# Step 9: Process all training and validation images
print("Processing all training images...")
process_images(train_images, "train")

print("Processing all validation images...")
process_images(val_images, "val")

# Step 10: Create a YAML file for the dataset
fish_dataset_yaml = f"""
train: "{base_path}/images/train"
val: "{base_path}/images/val"

# Number of classes
nc: 1

# Class names
names: ['Fish']
"""

# Save the YAML file
yaml_file_path = os.path.join(base_path, "fish_dataset.yaml")
with open(yaml_file_path, "w") as yaml_file:
    yaml_file.write(fish_dataset_yaml)

print(f"YAML file created: {yaml_file_path}")

# Step 11: Train the YOLOv8n model using the generated dataset
small_model = YOLO("yolov8n.pt")  # Load the smaller YOLOv8n model

# Train the model using the generated fish-only dataset
small_model.train(data=yaml_file_path, epochs=50, imgsz=416, batch=16, lr0=0.001)

print("Training complete!")
# Save the model
small_model.save(r"V:\temp\yolov8n_fish_trained_lgds.pt")

# Evaluate model performance
metrics = small_model.val(data=yaml_file_path)  # Evaluate precision, recall, and mAP

In [2]:
import os
import torch
from ultralytics import YOLO

unzipped_folder = r"V:\temp\20161014_192048_1"  # Use raw string for Windows paths

# Step 3: Check if CUDA is available and load the large model (YOLOv8x) to CUDA if possible
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load the large model (YOLOv8x) that was fine-tuned for VME detection
large_model = YOLO(r"V:\temp\models\best.pt")  # Update to your model path
large_model = large_model.to(device)  # Move the model to GPU (CUDA) if available

# Step 4: Create folders for the new dataset (images and labels)
base_path = r"V:\temp\lg_fish_dataset"


# Step 10: Create a YAML file for the dataset
fish_dataset_yaml = f"""
train: "V:/temp/lg_fish_dataset/images/train"
val: "V:/temp/lg_fish_dataset/images/val"

# Number of classes
nc: 1

# Class names
names: ['Fish']
"""

# Save the YAML file
yaml_file_path = os.path.join(base_path, "fish_dataset.yaml")
with open(yaml_file_path, "w") as yaml_file:
    yaml_file.write(fish_dataset_yaml)

print(f"YAML file created: {yaml_file_path}")


# Step 11: Train the YOLOv8n model using the generated dataset
small_model = YOLO("yolov8n.pt")  # Load the smaller YOLOv8n model

# Train the model using the generated fish-only dataset
small_model.train(data=yaml_file_path, epochs=50, imgsz=416, batch=16, lr0=0.001)

print("Training complete!")
# Save the model
small_model.save(r"V:\temp\yolov8n_fish_trained_lgds.pt")

# Evaluate model performance
metrics = small_model.val(data=yaml_file_path)  # Evaluate precision, recall, and mAP

Using device: cuda
YAML file created: V:\temp\lg_fish_dataset\fish_dataset.yaml
Ultralytics YOLOv8.2.95  Python-3.10.14 torch-2.4.1+cu118 CUDA:0 (Quadro P4000, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=V:\temp\lg_fish_dataset\fish_dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frame

100%|██████████| 755k/755k [00:00<00:00, 2.40MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning \\PICQUEENFISH\OPTICAL\temp\lg_fish_dataset\labels\train... 8466 images, 10262 backgrounds, 0 corrupt: 100%|██████████| 18728/18728 [00:16<00:00, 1107.69it/s]


train: New cache created: \\PICQUEENFISH\OPTICAL\temp\lg_fish_dataset\labels\train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                   all       4682       5335      0.773       0.76      0.838      0.655

50 epochs completed in 2.627 hours.
Optimizer stripped from runs\detect\train3\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train3\weights\best.pt, 6.2MB

Validating runs\detect\train3\weights\best.pt...
Ultralytics YOLOv8.2.95  Python-3.10.14 torch-2.4.1+cu118 CUDA:0 (Quadro P4000, 8192MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 147/147 [00:21<00:00,  6.97it/s]


                   all       4682       5335      0.779      0.758      0.838      0.655
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\train3
Training complete!
Ultralytics YOLOv8.2.95  Python-3.10.14 torch-2.4.1+cu118 CUDA:0 (Quadro P4000, 8192MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning \\PICQUEENFISH\OPTICAL\temp\lg_fish_dataset\labels\val.cache... 2721 images, 1961 backgrounds, 0 corrupt: 100%|██████████| 4682/4682 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 293/293 [00:22<00:00, 13.21it/s]


                   all       4682       5335       0.78      0.758      0.838      0.656
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\train32


### Start Model Metrics

In [12]:
metrics = small_model.val(data="/content/fish_dataset.yaml")  # This will evaluate precision, recall, and mAP


Ultralytics YOLOv8.2.93 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/fish_dataset/labels/val.cache... 337 images, 32 backgrounds, 0 corrupt: 100%|██████████| 369/369 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:04<00:00,  5.94it/s]


                   all        369        473      0.863      0.869      0.936      0.856
Speed: 0.1ms preprocess, 2.5ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train22


## Test Script

In [9]:
import os
import py7zr
import torch
from ultralytics import YOLO

# Step 1: Install Ultralytics YOLOv8 and py7zr (if not already installed)
#!pip install ultralytics py7zr

# Define the folder path
unzipped_folder = "/content/original/original"

# Step 3: Check if CUDA is available and load the large model (YOLOv8x) to CUDA if possible
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load the large model (YOLOv8x) that was fine-tuned for VME detection
large_model = YOLO("/content/best.pt")  # Replace with your model path
large_model = large_model.to(device)  # Move the model to GPU (CUDA) if available

# Step 4: Create folders for the new dataset (images and labels)
base_path = "/content/fish_dataset"

# Create directories for train/val images and labels
os.makedirs(f"{base_path}/images/train", exist_ok=True)
os.makedirs(f"{base_path}/images/val", exist_ok=True)
os.makedirs(f"{base_path}/labels/train", exist_ok=True)
os.makedirs(f"{base_path}/labels/val", exist_ok=True)

# Step 5: Define the input folder containing the original images (unzipped)
input_images_folder = unzipped_folder
image_paths = [os.path.join(input_images_folder, img) for img in os.listdir(input_images_folder) if img.endswith(('.jpg', '.png'))]

# Split images into train and validation (80% train, 20% validation)
train_images = image_paths[:int(0.8 * len(image_paths))]
val_images = image_paths[int(0.8 * len(image_paths)):]

# Function to save YOLO format labels (class_id x_center y_center width height)
def save_yolo_labels(label_path, class_id, bbox, image_width, image_height):
    if bbox is not None and len(bbox) > 0:  # Ensure there are bounding boxes
        x_center = (bbox[0] + bbox[2]) / 2 / image_width
        y_center = (bbox[1] + bbox[3]) / 2 / image_height
        width = (bbox[2] - bbox[0]) / image_width
        height = (bbox[3] - bbox[1]) / image_height
        with open(label_path, "a") as f:
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
        print(f"Label saved for {label_path}: {class_id} {x_center} {y_center} {width} {height}")
    else:
        print(f"No bounding boxes found for {label_path}, skipping label.")

# Function to process images, run inference, and save results
def process_images(image_paths, split):
    for image_path in image_paths:
        # Step 6: Run the large model to detect objects in the image (with GPU if available)
        results = large_model(image_path)

        # Check if any instances were detected
        print(f"Processing {image_path}, found {len(results[0].boxes)} instances.")

        # Extract image dimensions
        img_name = os.path.basename(image_path)
        img = results[0].orig_img
        img_height, img_width = img.shape[:2]

        # Save the image to the appropriate split folder (train/val)
        output_image_path = f"{base_path}/images/{split}/{img_name}"
        os.rename(image_path, output_image_path)

        # Step 7: Filter results to only include "fish" category (class 2 based on your provided example)
        fish_class_index = 2  # Update to class 2 for fish
        for result in results:
            for i, cls in enumerate(result.boxes.cls):
                if cls == fish_class_index:  # Only keep fish detections
                    bbox = result.boxes.xyxy[i].cpu().numpy()  # Bounding box (x1, y1, x2, y2)
                    print(f"Detected fish with bounding box: {bbox}")
                    print(f"Image dimensions: {img_width}x{img_height}")

                    # Step 8: Save label file in YOLO format
                    label_name = img_name.replace(".jpg", ".txt").replace(".png", ".txt")
                    label_path = f"{base_path}/labels/{split}/{label_name}"

                    # Save the bounding box in YOLO format (class_id = 0 for fish)
                    save_yolo_labels(label_path, class_id=0, bbox=bbox, image_width=img_width, image_height=img_height)

# Step 9: Process train and validation images (use a small sample for testing)
print("Processing a small sample of training images to test label generation...")
sample_images = train_images[:5]  # Test with a few images first
process_images(sample_images, "train")

# Check if labels were generated
train_labels = os.listdir(f"{base_path}/labels/train")
print(f"Generated labels for training set: {train_labels}")

print("Test complete!")


Using device: cuda
Processing a small sample of training images to test label generation...

image 1/1 /content/original/original/01649.jpg: 512x640 1 Fish, 69.2ms
Speed: 3.2ms preprocess, 69.2ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 640)
Processing /content/original/original/01649.jpg, found 1 instances.
Detected fish with bounding box: [     175.25      150.98      378.53      304.93]
Image dimensions: 968x728
Label saved for /content/fish_dataset/labels/train/01649.txt: 0 0.286039967182254 0.31312393356155566 0.21000108640056012 0.21146348806527945

image 1/1 /content/original/original/00381.jpg: 512x640 2 Fishs, 43.9ms
Speed: 2.2ms preprocess, 43.9ms inference, 1.4ms postprocess per image at shape (1, 3, 512, 640)
Processing /content/original/original/00381.jpg, found 2 instances.
Detected fish with bounding box: [      103.2      167.75      311.31      326.53]
Image dimensions: 968x728
Label saved for /content/fish_dataset/labels/train/00381.txt: 0 0.2141046

In [ ]:
## Windows CUDA Troubleshooting

In [2]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.version.cuda)  # Should show 11.8 or similar
print(f"Available GPUs: {torch.cuda.device_count()}")
print(f"Current GPU: {torch.cuda.current_device()}")
print(f"GPU Name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
# For Windows Troubleshooting 
# run via cmd to check cuda version
# nvcc --version
# uninstall 
#(yolo8n_model_train) C:\Users\PICHLMRUser>pip uninstall torch torchvision torchaudio
# reinstall
#(yolo8n_model_train) C:\Users\PICHLMRUser>pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

True
11.8
Available GPUs: 1
Current GPU: 0
GPU Name: Quadro P4000
